In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

%matplotlib inline

In [3]:
cafa3_dir = '/gpfs/alpine/bie108/proj-shared/cafa3/'

In [4]:
from itertools import groupby

def fasta_iter():
    """
    given a fasta file. yield tuples of header, sequence
    """
    with open(os.path.join(cafa3_dir, 'CAFA3_training_data/uniprot_sprot_exp.fasta'), 'r') as fh:
        faiter = (x[1] for x in groupby(fh, lambda line: line[0] == ">"))
        for header in faiter:
            headerStr = header.__next__()[1:].strip()
            seq = "".join(s.strip() for s in faiter.__next__())
            yield headerStr, seq
            
seq_df = pd.DataFrame(fasta_iter(), columns=['accession', 'sequence'])
seq_df.shape

(66841, 2)

In [5]:
go_terms = pd.read_csv(os.path.join(cafa3_dir, 'CAFA3_training_data/uniprot_sprot_exp.txt'), sep='\t', header=None).rename(
    columns={0: 'accession', 1:'GO TERM', 2:'GO ASPECT'})

go_terms.head()

,accession,GO TERM,GO ASPECT
0,P0DJZ0,GO:0030430,C
1,P32234,GO:0005525,F
2,P83011,GO:0043231,C
3,P83010,GO:0043231,C
4,P81928,GO:0007275,P


In [6]:
import sys
sys.path.append('../..')

from bert.dataset import encode
from bert.go import Ontology

ont = Ontology(restrict_terms=False, obo_file='go_cafa3.obo.gz')
ont.total_nodes

42819

In [8]:
import itertools
from tqdm import tqdm
tqdm.pandas()

In [9]:
all_terms = go_terms.groupby('accession').progress_apply(
    lambda df: pd.Series(list(ont.get_ancestors(df['GO TERM'].values))))

100%|██████████| 66841/66841 [02:00<00:00, 552.95it/s] 


In [14]:
len(all_terms.unique())

28474

In [18]:
ont_new = Ontology()
ont_new.total_nodes

32012

In [31]:
unique_terms = pd.Series(all_terms.unique())
missing = unique_terms[~unique_terms.isin(ont_new.term_index.values())]

len(missing)

1537

In [40]:
all_terms = all_terms.reset_index().drop('level_1', 1).rename(columns={0: 'GO TERM'})

In [ ]:
grouped_go_terms = go_terms.groupby('accession')['GO TERM'].apply(lambda x: x.values)

In [ ]:
# The following functions can be used to convert a value to a type compatible
# with tf.Example.

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _serialize_array(value):
    return _bytes_feature(tf.io.serialize_tensor(value))

def inputs_generator(swissprot_df):
    for _, row in tqdm(swissprot_df.iterrows(), total=len(swissprot_df)):
#        sequence = encode(row.sequence, max_seq_len)
        ancestor_array = ont.termlist_to_array(ont.get_ancestors(
            grouped_go_terms[row.accession]), dtype=int)
        
        features = {
            'sequence': _bytes_feature(row.sequence.encode('utf8')),
            'annotation': _serialize_array(ancestor_array)
        }
        
        example_proto = tf.train.Example(features=tf.train.Features(feature=features))
        yield example_proto.SerializeToString()

In [ ]:
# np.savez_compressed('uniref50_split.npz', train=train, valid=valid, test=test)
# pd.Series(ont.term_index).to_csv('term_index.csv', header=False)

In [ ]:
seq_df.shape

In [ ]:
seq_df.sample(frac=1., random_state=1)
valid, train = np.split(seq_df.accession, [4000,])

In [ ]:
import tensorflow as tf

In [ ]:
# Preprocess and write the train dataset to disk
serialized_train_dataset = tf.data.Dataset.from_generator(
    lambda: inputs_generator(seq_df[seq_df.accession.isin(train)]),
    output_types=tf.string, output_shapes=())

filename = os.path.join(cafa3_dir, 'tfrecords', 'go_train.tfrecord.gz')
writer = tf.data.experimental.TFRecordWriter(filename, compression_type='GZIP')
writer.write(serialized_train_dataset)


# Preprocess and write the valid dataset to disk
serialized_valid_dataset = tf.data.Dataset.from_generator(
    lambda: inputs_generator(seq_df[seq_df.accession.isin(valid)]),
    output_types=tf.string, output_shapes=())

filename = os.path.join(cafa3_dir, 'tfrecords', 'go_valid.tfrecord.gz')
writer = tf.data.experimental.TFRecordWriter(filename, compression_type='GZIP')
writer.write(serialized_valid_dataset)

In [ ]:
def parse_example(example):
    parsed = tf.io.parse_single_example(example, features={
        'sequence': tf.io.FixedLenFeature([], tf.string, default_value=''),
        'annotation': tf.io.FixedLenFeature([], tf.string, default_value=''),
    })
    
    sequence = encode(parsed['sequence'], 1024)
    annotation = tf.io.parse_tensor(parsed['annotation'], out_type=tf.int64)
    
    return sequence, annotation

cafa3_dir = '/gpfs/alpine/bie108/proj-shared/cafa3/'
train_dataset = tf.data.TFRecordDataset(
    os.path.join(cafa3_dir, 'tfrecords', 'go_train.tfrecord.gz'),
    compression_type='GZIP', num_parallel_reads=tf.data.experimental.AUTOTUNE)\
    .map(parse_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
    .repeat().shuffle(buffer_size=5000)\
    .padded_batch(batch_size=16,
                  padded_shapes=(([1024], [ont.total_nodes])))\
    .prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
class_sample = np.concatenate([b.numpy() for a,b in tqdm(train_dataset.take(1000))])
total = np.prod(class_sample.shape)
pos = class_sample.sum()
neg = total - pos
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

initial_bias = np.log(class_sample.sum(0) / class_sample.shape[0] + np.finfo(float).eps)

In [ ]:
np.save(os.path.join(cafa3_dir, 'tfrecords', 'bias.npy'), initial_bias)

In [ ]:
len(initial_bias)

In [ ]:
ont.total_nodes